# Set Up and Load Raw Data

In [1]:
!pip install -q scikit-multilearn transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

## Load Zipped Raw Data and Preprocessed Image Data From Google Drive

In [3]:
# only run this once!
# from google.colab import drive

# drive.mount('/content/drive')

In [4]:
import zipfile
import io

ZIP_FILE_PATH = "drive/MyDrive/personal_projects/movie_genre_prediction/clean_data.zip"
zf = zipfile.ZipFile(ZIP_FILE_PATH, "r")
zf.extractall()

In [5]:
big_data_df = pd.read_csv('raw_data/clean_data/clean_big_data.csv').drop(columns=['Unnamed: 0'])
big_data_df.head(1)

,imdb_id,genre,plot
0,tt1517268,"['Adventure', 'Comedy', 'Fantasy']",barbie suffers a crisis that leads her to ques...


In [6]:
image_array = np.load('raw_data/clean_data/clean_image_array.npy')
image_array.shape

(23140, 256, 256, 3)

In [7]:
# Double check if our image_array and big_data_df are the same size
assert big_data_df.shape[0] == image_array.shape[0]

## Preprocess Genres and Split to Train, Test, Validation Datasets

In [8]:
def preprocess_genre(df: pd.DataFrame):
    df["genre"] = df["genre"].apply(eval).apply(lambda x: [genre.strip() for genre in x])
    return df

In [9]:
big_data_df = preprocess_genre(big_data_df)
big_data_df.head(1)

,imdb_id,genre,plot
0,tt1517268,"[Adventure, Comedy, Fantasy]",barbie suffers a crisis that leads her to ques...


In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

# Multilabel_binarizer is fit to an array of list of labels
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(big_data_df['genre'])

#transform target variable
y = multilabel_binarizer.transform(big_data_df['genre'])
genre_names = multilabel_binarizer.classes_

# Adding the name of genres
for i in range(len(genre_names)):
    big_data_df[f"{genre_names[i]}"] = y[:,i]

print(y.shape, big_data_df.shape)

(23140, 22) (23140, 25)


In [11]:
big_data_df.head(1)

,imdb_id,genre,plot,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Music,Musical,Mystery,Reality-TV,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt1517268,"[Adventure, Comedy, Fantasy]",barbie suffers a crisis that leads her to ques...,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from skmultilearn.model_selection import iterative_train_test_split

def balanced_split(df: pd.DataFrame, labels: np.array, image_array: np.array, test_size=0.5):
    """
    index_array: an array of our current df index,
    iterative_train_test_split: a function that considers the distribution of possible labels when splitting
    """
    index_array = np.expand_dims(np.arange(len(df)), axis=1)
    train_index, y_train, test_index, y_test = iterative_train_test_split(index_array, labels, test_size)
    train_image_array, test_image_array = np.take(image_array, train_index.ravel(), axis=0), np.take(image_array, test_index.ravel(), axis=0)
    return df.iloc[train_index[:, 0]], train_image_array, y_train, df.iloc[test_index[:, 0]], test_image_array, y_test

In [13]:
# Split dataset to train and test_val (will split test and val again!)
train_df, train_image_array, y_train, test_val_df, test_val_image_array, y_test_val = balanced_split(big_data_df, y, image_array, 0.3)
print(
    train_df.shape,
    y_train.shape,
    train_image_array.shape,
    test_val_df.shape,
    test_val_image_array.shape,
    y_test_val.shape
    )

(16284, 25) (16284, 22) (16284, 256, 256, 3) (6856, 25) (6856, 256, 256, 3) (6856, 22)


In [14]:
# Split test_val further to test and val datasets!
test_df, test_image_array, y_test, val_df, val_image_array, y_val = balanced_split(test_val_df, y_test_val, test_val_image_array)
print(
    test_df.shape,
    test_image_array.shape,
    y_test.shape,
    val_df.shape,
    val_image_array.shape,
    y_val.shape
    )

(3428, 25) (3428, 256, 256, 3) (3428, 22) (3428, 25) (3428, 256, 256, 3) (3428, 22)


**RECAP**

Let's recap what we have up to this point!
Balanced train, test, validation datasets consisting of:
1. Dataframes with imdb_id and plot (train_df, test_df, val_df)
2. Preprocessed image_arrays (train_image_array, test_image_array and val_image_array)
3. Labels! (y_train, y_test, y_val)

# Text Preprocessing

In [15]:
from transformers import AutoTokenizer

def tokenize_encode_multimodal(df):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    text = df['plot'].to_list()
    encodings = tokenizer(text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
    return encodings['input_ids']

In [16]:
X_train_text = tokenize_encode_multimodal(train_df)
X_test_text = tokenize_encode_multimodal(test_df)
X_val_text = tokenize_encode_multimodal(val_df)

In [17]:
X_train_img = train_image_array
X_test_img = test_image_array
X_val_img = val_image_array

# RESNET 50

In [18]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [19]:
def load_model():
    model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    return model


In [20]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [21]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = Flatten()
    dense_layer = Dense(500, activation='relu')
    prediction_layer = Dense(512, activation='relu')

    model = Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model

# BERT Model

In [22]:
from transformers import TFBertModel
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input

In [23]:
# define the BERT-based text feature extractor
def build_text_model():
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    inputs = Input(shape=(None,), dtype=tf.int32, name='input_word_ids')
    outputs = bert_model(inputs)[1]
    text_model = Model(inputs=inputs, outputs=outputs)
    return text_model

# Multimodal

In [24]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
# define the multimodal document classification model
def build_multimodal_model(num_classes):
    model = load_model()
    model = set_nontrainable_layers(model)
    img_model = add_last_layers(model)
    text_model = build_text_model()
    img_input = Input(shape=(256, 256, 3), name='img_input')
    text_input = Input(shape=(None,), dtype=tf.int32, name='text_input')
    img_features = img_model(img_input)
    text_features = text_model(text_input)
    concat_features = concatenate([img_features, text_features])
    x = Dense(512, activation='relu')(concat_features)
    x = Dense(num_classes, activation='sigmoid')(x)
    multimodal_model = Model(inputs=[img_input, text_input], outputs=x)
    return multimodal_model

In [26]:
num_classes = y_train.shape[1]
multimodal_model = build_multimodal_model(num_classes)
multimodal_model.summary()

94765736/94765736 [==============================] - 4s 0us/step


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_input (InputLayer)      [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 text_input (InputLayer)     [(None, None)]               0         []                            
                                                                                                  
 sequential (Sequential)     (None, 512)                  8938072   ['img_input[0][0]']           
                                                          4                                       
                                                                                                  
 model (Functional)          (None, 768)                  1094822   ['text_input[0][0]']    

In [27]:
legacy_adam = tf.keras.optimizers.legacy.Adam()

In [28]:
# compile the model and train on the train set
multimodal_model.compile(optimizer=legacy_adam, loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'binary_accuracy', 'categorical_accuracy'])

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 10,
                   verbose = 1,
                   restore_best_weights = True)

history = multimodal_model.fit([(X_train_img, X_train_text)], tf.convert_to_tensor(y_train), epochs=5, batch_size=16, validation_data=([(X_val_img, X_val_text)], tf.convert_to_tensor(y_val)), callbacks = [es])

Epoch 1/5
1018/1018 [==============================] - 578s 544ms/step - loss: 0.3812 - accuracy: 0.2627 - auc: 0.7588 - binary_accuracy: 0.8824 - categorical_accuracy: 0.2627 - val_loss: 0.3012 - val_accuracy: 0.3086 - val_auc: 0.7924 - val_binary_accuracy: 0.8870 - val_categorical_accuracy: 0.3086
Epoch 2/5
1018/1018 [==============================] - 558s 548ms/step - loss: 0.2853 - accuracy: 0.3233 - auc: 0.8200 - binary_accuracy: 0.8928 - categorical_accuracy: 0.3233 - val_loss: 0.3070 - val_accuracy: 0.2812 - val_auc: 0.7887 - val_binary_accuracy: 0.8870 - val_categorical_accuracy: 0.2812
Epoch 3/5
1018/1018 [==============================] - 558s 548ms/step - loss: 0.2585 - accuracy: 0.3838 - auc: 0.8608 - binary_accuracy: 0.9010 - categorical_accuracy: 0.3838 - val_loss: 0.3124 - val_accuracy: 0.3063 - val_auc: 0.7942 - val_binary_accuracy: 0.8861 - val_categorical_accuracy: 0.3063
Epoch 4/5
1018/1018 [==============================] - 558s 548ms/step - loss: 0.2276 - accuracy:

# Comparing predictions with y_test!

In [36]:
y_test

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0]])

In [37]:
y_pred = multimodal_model.predict([X_test_img, X_test_text])

108/108 [==============================] - 40s 357ms/step


In [69]:
def round_up_predictions(y_pred, threshold):
    holder = np.zeros(y_pred.shape, dtype=np.int8)
    for i in range(len(y_pred)):
        holder[i] = np.array([1 if pred > threshold else 0 for pred in y_pred[i]], dtype=np.int8)
    return holder
y_pred_binary = round_up_predictions(y_pred, 0.3)

In [98]:
def all_correct(y_pred_binary, y_test):
    assert y_pred_binary.shape == y_test.shape
    ans = []
    for i in range(y_pred_binary.shape[0]):
        if np.all(y_pred_binary[i] == y_test[i]):
            ans.append(1)
        else:
            ans.append(0)
    return np.average(np.array(ans))
print(all_correct(y_pred_binary, y_test))

0.024795799299883313


In [108]:
def f_all_correct(threshold):
    y_pred_binary = round_up_predictions(y_pred, threshold)
    return -1 * all_correct(y_pred_binary, y_test)

In [109]:
from scipy.optimize import minimize_scalar

In [110]:
minimize_scalar(f_all_correct, bounds=(0, 1))

 message: Solution found.
 success: True
  status: 0
     fun: -0.026837806301050177
       x: 0.3798226527775452
     nit: 19
    nfev: 19

So, if we set the threshold to 0.3798, we get a "all_correct" accuracy of 0.026!

In [111]:
def one_correct(y_pred_binary, y_test):
    assert y_pred_binary.shape == y_test.shape
    ans = []
    for i in range(y_pred_binary.shape[0]):
        y_pred_i = y_pred_binary[i]
        y_test_i = y_test[i]
        appended = False
        for j in range(y_pred_i.shape[0]):
            if y_pred_i[j] == 1 and y_test_i[j] == 1 and not appended:
                ans.append(1)
                appended = True
        if not appended:
            ans.append(0)
    assert y_pred_binary.shape[0] == len(ans)
    return np.average(np.array(ans))
print(one_correct(y_pred_binary, y_test))

0.7908401400233372


In [112]:
def f_one_correct(threshold):
    y_pred_binary = round_up_predictions(y_pred, threshold)
    return -1 * one_correct(y_pred_binary, y_test)

In [113]:
minimize_scalar(f_one_correct, bounds=(0, 1))

 message: Solution found.
 success: True
  status: 0
     fun: -0.9988331388564761
       x: 0.0011907108069308574
     nit: 24
    nfev: 24

## Conclusions:
Okay, we need an actual metric. Not just "one correct" or "all correct". Because adjusting the threshold could definitely allow us to get extremely high scores for one metric and sacrificing the other metric.

One correct: x = 0.001, p = 0.9988
All correct: x = 0.3789, p = 0.0268